In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import xgboost as xgb

In [2]:
df = pd.read_csv("new_train.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


Let's found our categorical columns and our numerical columns don't forget to remove the target Survived

In [3]:
categorical_columns = [col for col in df.columns if
                       df[col].dtype == "object"]
numerical_columns = [col for col in df.columns if
                     df[col].dtype in ["int64", "float64"]]

numerical_columns.remove("Survived")

categorical_columns, numerical_columns

(['Name', 'Sex', 'Ticket', 'Embarked'],
 ['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'])

Instantiate the Pipelines

In [4]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy = "constant", fill_value = "missing")),
    ("onehot", OneHotEncoder(handle_unknown = "ignore"))
])

numerical_transformer = Pipeline(steps = [
    ("imputer", SimpleImputer(strategy = "mean"))
])

In [5]:
preprocessor = ColumnTransformer(transformers = [
    ("cat", categorical_transformer, categorical_columns),
    ("num", numerical_transformer, numerical_columns)
])

In [6]:
clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", xgb.XGBClassifier())
])

Let's split our data into train an test. Put shuffle = False, because we will use another notebook to do the optimization

In [7]:
X = df.drop("Survived", axis = 1)
y = df.Survived

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, shuffle = False)

Don't worry the warning isn't important

In [8]:
clf.fit(X_train, y_train)

[13:43:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Beauj\git_projects\kaggle\env\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Name', 'Sex', 'Ticket',
                                                   'Embarked']),
                                                 ('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['PassengerId', 'Pclass',

In [9]:
clf.score(X_test, y_test)

0.8324022346368715

Save the model

In [10]:
import pickle

pickle.dump(clf, open("xgb_clf_model.pkl", "wb"))